In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Encoding Functions

In [ ]:
aa_idx = {'A':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'K':9, 
            'L':10, 'M':11, 'N':12, 'P':13, 'Q':14, 'R':15, 'S':16, 'T':17, 
            'V':18, 'W':19, 'Y':20, '-':21}

def onehot(seq, k=15):
  s = list(seq)
  if (len(s) < k):
    s = s + (['-'] * (k - len(s)))
  else:
    s = s[0:k]
  
  vec = []
  for let in range(k):
    char = s[let]
    row = np.random.uniform(low=0.001, high=0.01,size=21)
    if (char not in aa_idx):
      char = '-'
    row[aa_idx[char]-1] = 1
    vec.append(row)
  vec = np.array(vec).flatten()
  return vec

def softhot(seq, k=15):
  s = list(seq)
  if (len(s) < k):
    s = s + (['-'] * (k - len(s)))
  else:
    s = s[0:k]
  
  vec = []
  for let in range(k):
    char = s[let]
    row = np.ones((21)) * ((0.1)/(20)) 
    if (char not in aa_idx):
      char = '-'
    row[aa_idx[char]-1] = 0.9
    vec.append(row)
    if (char == '-'):
      row = np.ones((21)) * (-400000)
  vec = np.array(vec).flatten()
  return vec

# met refers to function
def encode_seq(method, peptide_seqs, k=15):
  methods_dict = {'ONEHOT':onehot, 'BLOSUM':blosum, 'SOFTHOT':softhot}
  met = methods_dict[method]

  if (k == 15):
    mapped = peptide_seqs.apply(lambda x: met(x))
  else:
    mapped = peptide_seqs.apply(lambda x: met(x, k))
  return mapped

def pseudo_HLA(seq, ind):
  seq = list(seq)
  N = [ seq[i] for i in ind]
  sequence = ''.join(N)
  print(seq, sequence)
  return sequence

# Training Driver

In [ ]:
def prep_training_full(y_cat, hla_encoding_method="pseudo"):
  
  path = "/content/drive/MyDrive/02-518/02518 Comp Medicine Project/Input Data Files"
  
  final_df = pd.DataFrame(columns=["Sequence", "HLA", "Pep_Length","Y_val"])

  if (y_cat == "binding"):
    input_path = ("%s/binding_dat.csv" % (path))
  else:
    input_path = ("%s/immunogenicity_dat.csv" % (path))

  input_df = pd.DataFrame(pd.read_csv(input_path, index_col=0))

  final_df["Sequence"] = input_df["Peptide"]
  final_df["Pep_Length"] = input_df["Length"]

  HLA_pseudo = input_df["HLA"].apply(lambda x: HLA_DICT[x]['HLA_Pseudo'])
  
  if (hla_encoding_method == "pseudo"):
    final_df["HLA"] = input_df["HLA"]
  else:
    final_df["HLA"] = HLA_pseudo

  if (y_cat == "binding"):
    final_df["Y_val"] = input_df["IC50_transformed"]
  else:
    final_df["Y_val"] = input_df["Immunogenic"]

  return final_df

def prep_training(y_cat,hla_encoding_method):
  df = prep_training_full(y_cat, hla_encoding_method)
  X = df[["Sequence", "HLA", "Pep_Length"]]
  y = df["Y_val"]

  return X, y

In [ ]:
# pre-processed data formatted as [sequence, hla, pep_length]

from collections import namedtuple
from tensorflow.keras.utils import Sequence

DATA_ENTRY = namedtuple("data_entry_ic50", "sequence, hla, y_val, pep_length")

class DataGenerator(Sequence):
  def __init__(self, batch_size, samples, y_cat):
        self.batch_size = batch_size
        self.samples = samples
        self.y_cat = y_cat
        self.epitope_enc_map = {}
        self.hla_enc_map = {}
        self.init_data()
        self.on_epoch_end()

  def init_data(self):
    for sample in self.samples:
      sequence=sample[0]
      hla=sample[1]
      pep_length=int(sample[2])
      self.epitope_enc_map[sequence] = (onehot(sequence, 9)).reshape(1, 9, 21)
      self.hla_enc_map[hla] = (onehot(hla, 34)).reshape(1, 34, 21)

  def __len__(self):
    return (np.ceil(len(self.samples) / float(self.batch_size))).astype(np.int)

  def on_epoch_end(self):
    #'Updates indexes after each epoch'
    np.random.shuffle(self.samples)

  def __getitem__(self, idx):
    hla_alleles = []
    epitopes = []
    y_vals = []
    batch_sample = self.samples[idx * self.batch_size : (idx+1) * self.batch_size]

    i = 0
    for sample in batch_sample:

      y_val= float(sample[3])
      sequence = sample[0]
      hla = sample[1]

      # protein
      epitope_encoded = self.epitope_enc_map[sequence]
      epitopes.append(epitope_encoded)

      # hla
      hla_encoded = self.hla_enc_map[hla]
      hla_alleles.append(hla_encoded)

      # log_ic50 or binder versus non-binder or immunogenic versus non-immunogenic
      if (self.y_cat != "binding"):
        y_vals.append(int(y_val))
      else:
        y_vals.append(y_val)

      i += 1

    ret_in = {'epitope':np.array(epitopes), 'hla':np.array(hla_alleles)}
    ret_out = np.array(y_vals)
      
    return (ret_in, [ret_out],)

# **Convolutional Neural Net Code**

In [ ]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.utils import plot_model
from keras.regularizers import *

def vgg_pool_block(layer, num_filter, max_pool=True):
    # a block does a convolution, batch norm, and max pooling
    layer = Conv2D(num_filter,
           (1, 3),
           strides=(1, 1),
           kernel_regularizer=l2(l=0.1),
           data_format='channels_last',
           padding='same')(layer)

    layer = BatchNormalization()(layer)
    
    layer = LeakyReLU()(layer)
    
    layer = Conv2D(num_filter,
           (1, 3),
           strides=(1, 1),
           kernel_regularizer=l2(l=0.1),
           data_format='channels_last',
           padding='same')(layer)

    layer = BatchNormalization()(layer)
    
    layer = LeakyReLU()(layer)
    
    if max_pool:
        layer = MaxPooling2D(pool_size=(1,2))(layer)
    
        layer = Dropout(0.5)(layer)

    return layer

# we'll use num_blocks = 3 for this project
def vgg_net(input_data, num_blocks=3, max_pool=True):
    num_filter = 64
    data = vgg_pool_block(input_data, num_filter, max_pool=max_pool)
    
    # run convolution blocks
    for i in range(2, num_blocks+1):        
        num_filter *= 2
        data = vgg_pool_block(data, num_filter, max_pool=max_pool)
    
    return LeakyReLU()(data)

# **VGGNet Code**

In [ ]:
def epitope_net(input_data):
    num_filter = 64
    data = vgg_pool_block(input_data, num_filter)
    data = LeakyReLU()(data)
    
    info_channels = 10
    data = Conv2D(info_channels,
                  (1, 1),
                  strides=(1, 1),
                  use_bias=False,
                  kernel_regularizer=l2(l=0.1),
                  kernel_initializer=glorot_uniform(),
                  padding='valid')(data)

    data = BatchNormalization()(data)
    data = LeakyReLU()(data)
    return data

def protein_net(input_data):
    num_filter = 64
    data = vgg_pool_block(input_data, num_filter)
    data = vgg_pool_block(data, num_filter*2)
    
    info_channels = 10
    data = Conv2D(info_channels,
                  (1, 1),
                  strides=(1, 2),
                  use_bias=False,
                  kernel_regularizer=l2(l=0.1),
                  kernel_initializer=glorot_uniform(),
                  padding='valid')(data)
    data = BatchNormalization()(data)
    data = LeakyReLU()(data)
    return data

# ResNet Code

In [ ]:
'''
Now we can start with the ResNet code

The idea of a Residual network is that the input to each convolutional block is added as a "residual" to its output. 
In other words, information from earlier parts of the network "skips" to later parts of the network.

We use the same overall design as vgg, but making each convolutional block into a residual block.
'''

from keras import Sequential, Model
from keras.activations import relu
from keras.layers import *

class Residual(Model):  #@save
    """The Residual block of ResNet."""
    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = Conv2D(num_channels, padding='same', kernel_size=3, strides=strides)
        self.conv2 = Conv2D(num_channels, kernel_size=3, padding='same')
        self.conv3 = None
        if use_1x1conv:
            self.conv3 = Conv2D(num_channels, kernel_size=1, strides=strides)
        self.bn1 = BatchNormalization()
        self.bn2 = BatchNormalization()
        self.num_channels = num_channels

    def call(self, X):

        Y = relu(self.bn1(self.conv1(X)))
        
        Y = self.bn2(self.conv2(Y))
        
        if self.conv3 is not None:
            X = self.conv3(X)
        
        Y += X
        
        return relu(Y)

class ResnetBlock(Layer):
    def __init__(self, num_channels, num_residuals, first_block=False,
                 **kwargs):
        super(ResnetBlock, self).__init__(**kwargs)

        self.num_channels = num_channels
        self.num_residuals = num_residuals
        self.first_block = first_block
        
        self.residual_layers = []
        for i in range(num_residuals):
            if i == 0 and not first_block:
                self.residual_layers.append(
                    Residual(num_channels, use_1x1conv=True, strides=2))
            else:
                self.residual_layers.append(Residual(num_channels))

    def get_config(self):
        
        config = super().get_config().copy()
        config.update({
            #'residual_layers': self.residual_layers,
            'num_channels': self.num_channels,
            'num_residuals': self.num_residuals,
            'first_block': self.first_block
        })
        
        return config

    def call(self, X):
        for layer in self.residual_layers.layers:
            X = layer(X)
        return X

def res_net(small=False):
    # 3 convolutional blocks hardcoded
    return Sequential([
        Conv2D(64, kernel_size=(1,3), strides=(1,1), padding='same'),
        BatchNormalization(),
        Activation('relu'),
        MaxPool2D(pool_size=(1,2), strides=(1,1), padding='same'),
        # created earlier
        ResnetBlock(64, 2, first_block=True),
        ResnetBlock(128, 2),
        ResnetBlock(256, 2),
        GlobalAvgPool2D(),
        Dense(units=10)])

def res_net_new(x, small=False):

    
    x = Conv2D(64, kernel_size=(1,3), strides=(1,1), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size=(1,2))(x)
    
    
    x = ResnetBlock(64, 2, first_block=True)(x)
    x = ResnetBlock(128, 2)(x)
    
    if (not small):
        x = ResnetBlock(256, 2)(x)
        
        
    x = GlobalAvgPool2D()(x)
    
    x = Dense(units=10)(x)
    
    return x

# Context Extraction

In [ ]:
# extract features from the data

def context_extract_net(x):
    
    x = ZeroPadding2D((0, 2), data_format='channels_last')(x)

    x = LocallyConnected2D(512, 
                           (1, 2),
                           strides=(1, 1),
                           use_bias=False,
                           kernel_regularizer=l2(l=0.1),
                           kernel_initializer=glorot_uniform(),
                           padding='valid')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)

    x = LocallyConnected2D(512, 
                           (1, 2),
                           strides=(1, 1),
                           use_bias=False,
                           kernel_regularizer=l2(l=0.1),
                           kernel_initializer=glorot_uniform(),
                           padding='valid')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    
    x = MaxPooling2D(pool_size=(1, 2))(x)
    x = Dropout(0.5)(x)
    x = Flatten()(x)
    return x

# configure the convolutional model
def set_model(flag):

    seq_len = 34
    ligand_len = 9

    pseudo_input = Input(shape=(1, seq_len, 21), name="hla")
    ligand_input = Input(shape=(1, ligand_len, 21), name="epitope")
    
    if flag=='res':
        
        pseudo_net = res_net_new(pseudo_input)
        
        ligand_net = res_net_new(ligand_input, small=True)
    else: # run vgg
        pseudo_net = protein_net(pseudo_input)
        
        # use max pooling for ligand
        ligand_net = epitope_net(ligand_input)
            
    merge = concatenate([pseudo_net, ligand_net])
    
    if flag == 'vgg':
        # we only want this for vgg
        context = context_extract_net(merge)
    else:
        context = merge
        
    data = Dropout(0.5)(context)
    data = Dense(100,
              name="regression_dense",
              kernel_regularizer=l2(l=0.1),
              use_bias=True)(data)

    data = LeakyReLU()(data)
    data = Dropout(0.5)(data)

    output = Dense(1,
                   name="regression",
                   kernel_regularizer=l2(l=0.1),
                   use_bias=False,
                   activation="sigmoid")(data)
    #print("dense:", output.shape)
    model = Model(inputs=[pseudo_input, ligand_input], outputs=[output])
    return model

In [ ]:
def model_vgg():
    model = set_model('vgg')
    return model

def model_resnet():
    model = set_model("res")
    return model

# Training

In [ ]:
'''
Now we train the convolutional model of our choice, specified by 'flag'
'''
def train_model(data, flag):
  net = set_model(flag)
  trained = net.fit(data)
  return trained

# Cross-validation

In [ ]:
def split_set(samples, ratio):
  for _ in range(10):
    np.random.shuffle(samples)
  
  train_count = math.ceil(len(samples) * (1 - ratio))
  return samples[:train_count], samples[train_count:]

def to_np(arr):
  X = []
  for x in arr:
    X.append(x)
  X = np.array(X)
  return X

In [ ]:
import keras
from sklearn.model_selection import KFold
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import Sequence
from keras.initializers import glorot_uniform
from sklearn.metrics import roc_auc_score, f1_score, precision_score, mean_squared_error, r2_score


## NN Model is a neural net model 
## training data in form of a df with 4 columns: Sequence, HLA, Y_val, Pep_Length
## y --  either immunogenicity or binding scores
## y_cat = either binding or else
## outfile where you want to output data to
## hla_encoding should be pseudo and NEVER full sequence
def train_gen_crossval(nn_model, training_data, y_cat, out_file, hla_encoding_method="pseudo"):

  f = open(out_file, "w")
  # Split data into training, test sets
  splits = list(KFold(n_splits=5, shuffle=True, random_state=20).split(training_data))

  print('we splitted')
    
  for (i, (train_index, test_index)) in enumerate(splits):
    keras.backend.clear_session()
    # get train_data, validation_data

    trains = training_data.iloc[train_index,:].values
    train_samples, validation_samples = split_set(trains, 0.2)
    print("split training set")
    
    test_samples = training_data.iloc[test_index,:].values
    print("got test samples")
  
    # make generator out of train data, batch size=16
    train_generator = DataGenerator(256, train_samples, y_cat)
    print("made training generator:", train_generator)
  
    # make generator out of test data, batch size =16
    validation_generator = DataGenerator(256, validation_samples, y_cat)
    print("made validation generator:", validation_generator)

    # call model()
    model = nn_model()
    
    print("made model")
    # compile model
    if (y_cat == 'binding'):
      model.compile(optimizer=Adam(lr=0.001), loss=["mean_squared_error"], metrics = ['mean_squared_error'])
    
    else:
      model.compile(optimizer=Adam(lr=0.001), loss=['binary_crossentropy'],
                    metrics=[keras.metrics.BinaryAccuracy(), 
                             keras.metrics.TruePositives(),
                             keras.metrics.TrueNegatives(), 
                             keras.metrics.FalsePositives(),
                             keras.metrics.FalseNegatives()])

    weights_path = './' + 'weights_{}.h5'.format(i)  
    print("Model compiled")
    # checkpoint
    ckpt = ModelCheckpoint(weights_path, save_best_only=True, 
                           save_weights_only=False, verbose=0, 
                           monitor='val_loss', mode='auto')
    early = EarlyStopping(monitor='val_loss', patience=5, verbose=0, min_delta=0.001)
    
    model.train_generator = train_generator

    model.fit(model.train_generator, epochs=5, 
              steps_per_epoch=len(train_generator), 
              validation_steps=len(validation_generator),
              validation_data=validation_generator, 
              callbacks=[ckpt, early])
    # loads the best weights saved by the checkpoint
    model.load_weights(weights_path)

    hlas = []
    epitopes = []
    ys = []
    xs = []

    for sample in test_samples:
      hla_encoded = onehot(sample[1], 34).reshape(1, 34, 21)
      hlas.append(hla_encoded)
      epitope_encoded = onehot(sample[0], 9).reshape(1, 9, 21)
      epitopes.append(epitope_encoded)
      ys.append(sample[3])

    if (y_cat == "binding"):
        results = model.predict({'epitope': np.array(epitopes),
                             'hla': np.array(hlas)})
        results = np.array(results)
        print(results.shape)

        # write results
        reals = []
        preds = []
        bin_reals = []
        bin_preds = []

        for i in range(len(ys)):
            real = ys[i]
            reals.append(ys[i])
            predict_y = results[i]
            preds.append(predict_y)
            if (real < 0.4257):
                bin_reals.append(0)
            else:
                bin_reals.append(1)
            if (predict_y < 0.4257):
                bin_preds.append(0)
            else:
                bin_preds.append(1)

        reals = np.array(reals)
        preds = np.array(preds)
        bin_reals = np.array(bin_reals)
        bin_preds = np.array(bin_preds)

        auc = roc_auc_score(bin_reals, preds)
        f1 = f1_score(bin_reals, bin_preds)
        ppv = precision_score(bin_reals, bin_preds, pos_label=1)
        mse = mean_squared_error(reals, preds)
        r2 = r2_score(reals, preds)

        f.write("AUC: {},F1: {}, PPV:{}, MSE:{},R2: {}\n".format(auc, f1, ppv, mse, r2))

        print("Wrote scores for fold %d" % i)
    
    else: #immunogenicity
        print("testing immunogenicity")
        for y in ys: 
            xs.append(1)
        print((np.array(xs)).shape)
        results = model.evaluate({'epitope': np.array(epitopes),
                             'hla': np.array(hlas)}, np.array(ys), verbose=1)
        loss = results[0]
        auc = results[1]
        f.write("Cross Entropy: {}, AUC: {}".format(loss, auc))
        print(f'Cross Entropy: {loss}, AUC: {auc}\n')
        print("Wrote scores for fold %d" % i)

    print("Finish fold {}...".format(i))
    print('\n'*8)


# Train on its Own

In [ ]:
def train_gen(nn_model, training_data, y_cat, mask=False):
    print("starting training")
    keras.backend.clear_session()
    # get train_data, validation_data

    trains = training_data.values
    train_samples, validation_samples = split_set(trains, 0.2)
    print("split training set")
  
    # make generator out of train data, batch size=16
    train_generator = DataGenerator(256, train_samples, y_cat)
    print("made training generator:", train_generator)
  
    # make generator out of test data, batch size =16
    validation_generator = DataGenerator(256, validation_samples, y_cat)
    print("made validation generator:", validation_generator)

    # call model()
    model = nn_model()
    
    print("made model")
        # compile model
    if (y_cat == 'binding'):
        model.compile(optimizer=Adam(lr=0.001), loss=["mean_squared_error"], metrics = ['mean_squared_error'])
    else:
        model.compile(optimizer=Adam(lr=0.001), loss=['binary_crossentropy'], metrics=[keras.metrics.AUC()])

    weights_path = './' + 'best_lstm_imm_model.h5'
    print("Model compiled")
    # checkpoint
    ckpt = ModelCheckpoint(weights_path, save_best_only=True,save_weights_only=False, verbose=0, monitor='val_loss', mode='auto')
    
    early = EarlyStopping(monitor='val_loss', patience=5, verbose=0, min_delta=0.001)
        
    model.train_generator = train_generator
    
    print("Fitting Model..........")
    model.fit(model.train_generator, epochs=5, steps_per_epoch=len(train_generator), validation_steps=len(validation_generator),
              validation_data=validation_generator, callbacks=[ckpt, early])
        
    # loads the best weights saved by the checkpoint
    # model.load_weights(weights_path)

    
    print('\n'*8)
    print("Done! :D")
    return model

In [ ]:
df_train_binding=pd.read_csv("../input/binding-train-pseudo/binding_train_pseudo.csv",index_col=0)

In [ ]:
df_train_binding = df_train_binding[df_train_binding['Pep_Length'] <= 9]

# Call Data Generation/Training Function for VGG Model

In [ ]:
train_gen_crossval(model_vgg, df_train_binding, "binding", "test_model_output.txt", hla_encoding_method="pseudo")

In [ ]:
train_gen_crossval(model_resnet, df_train_binding, "binding", "test_model_output_resnet.txt", hla_encoding_method="pseudo")

# **Debug immunogenicity**

In [ ]:
df_train_immuno = pd.read_csv("../input/immunogenicity-train-pseudo/immunogenicity_train_pseudo.csv",index_col=0)
df_train_immuno = df_train_immuno[df_train_immuno['Pep_Length'] <= 9]

In [ ]:
train_gen_crossval(model_vgg, df_train_immuno, "immunogenicity", "immuno_model_output.txt", hla_encoding_method="pseudo")

In [ ]:
train_gen_crossval(model_resnet, df_train_immuno, "immunogenicity", "immuno_resnet_output_final.txt", hla_encoding_method="pseudo")

# Train ResNet

In [ ]:
train_gen(model_resnet, df_train_binding, "binding")

In [ ]:
def test(model_path, test_data, out_file):
    ys = []
    epitopes = []
    hlas = []

    model = model_resnet()

    test_samples = test_data.values

    model.load_weights(model_path)

    for sample in test_samples:
        hla_encoded = onehot(sample[1], 34).reshape(1,34, 21)
        hlas.append(hla_encoded)
        epitope_encoded = onehot(sample[0], 9).reshape(1,9, 21)
        epitopes.append(epitope_encoded)
        ys.append(sample[3])
        #print(ys)

    results = model.predict({'epitope': np.array(epitopes),'hla': np.array(hlas)})
    results = np.array(results)

    # write results
    preds = []
    bin_reals = []
    bin_preds = []


    for i in range(len(ys)):
        real=ys[i]
        predict_y = results[i]
        print(ys[i])
        preds.append(predict_y)
        if (real < 0.4257):
            bin_reals.append(0)
        else:
            bin_reals.append(1)
        if (predict_y < 0.4257):
            bin_preds.append(0)
        else:
            bin_preds.append(1)

    preds = np.array(preds)
    bin_reals = np.array(bin_reals)
    bin_preds = np.array(bin_preds)

    auc = roc_auc_score(bin_reals, preds)
    f1 = f1_score(bin_reals, bin_preds)
    ppv = precision_score(bin_reals, bin_preds, pos_label=1)
    print("AUC: {},F1: {}, PPV:{}\n".format(auc, f1, ppv))

    f.write("AUC: {}, F1: {}, PPV:{}\n".format(auc, f1, ppv))

In [ ]:
path = './best_lstm_imm_model.h5'
#path = "../output/kaggle/working/weights_4.h5"
df_dbpep_high = pd.DataFrame(pd.read_csv("../input/binding-train-pseudo/binding_train_pseudo.csv", index_col=0))
test(path, df_dbpep_high, "resnet_test.txt")

In [ ]:
path = './best_lstm_imm_model.h5'
#path = "../output/kaggle/working/weights_4.h5"
df_dbpep_high = pd.DataFrame(pd.read_csv("../input/immunogenicity-train-pseudo/immunogenicity_train_pseudo.csv", index_col=0))
test(path, df_dbpep_high, "resnet_test_immuno.txt")

In [ ]:
from keras.models import *
from keras.utils.generic_utils import CustomObjectScope

def main(binding_model, immunogenicity_model, out_file, test_X, test_y=None, 
         hla_encoding_method="pseudo", attention=False):
  if (attention):
    with CustomObjectScope({'Attention': Attention}):
      BA_model = load_model(binding_model)
      IMM_model = load_model(immunogenicity_model)
  else:
    BA_model = load_model(binding_model)
    IMM_model = load_model(immunogenicity_model)

  outfile = open("%s.txt" % (out_file), "r")
  for sample in test_X:
    hla = onehot(sample[0])
    peptide = onehot(sample[1])
    bind_out = BA_model.predict({
        'protein': np.array([hla]),
        'ligand': np.array([peptide]),
        })
    imm_out = IMM_model.predict({
      'protein': np.array([hla]),
      'ligand': np.array([peptide]), 
    })
    outfile.write('{},{},{} (log_ic50),{} (binary)'.format(sample[0], sample[1], out[0][0][0], out[1][0][0]))